In [1]:
# %pip install opencv-python numpy colour-science

In [2]:
# %pip install --upgrade colour-science

In [3]:
import os
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import cv2
import colour
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from skimage.color import rgb2lab, deltaE_ciede2000
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from colour import MSDS_CMFS, SDS_ILLUMINANTS, sd_to_XYZ, SpectralDistribution
from colour.models import RGB_COLOURSPACE_sRGB

In [4]:
# ====== TEST SET CONFIGURATION ======
TEST_SET_DIR = "test_results_model"
DEFICIENCY_TYPES = ["protanopia", "deuteranopia", "tritanopia"]
GLASS_OUTPUT_DIR = "test_results_glass"
CATEGORY_COMPARE_DIR = "category_comparison"
METHODS = ["daltonized", "glass_effect"]


def load_image(path):
    """Load and preprocess image to RGB float32 (0-1 range)"""
    if not os.path.exists(path):
        raise FileNotFoundError(f"Image not found: {path}")

    img = cv2.imread(path)
    if img is None:
        raise ValueError(f"Failed to load image: {path}")

    # Convert to float32 in 0-1 range and RGB format
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 255.0
    return img

In [5]:
# Actual spectral transmittance data from CSV
wavelengths = np.array([
    380.5685945444981, 383.532872207571, 385.5610621875682, 387.2772229398735,
    388.21331062294917, 388.99338369217884, 390.24150060294636, 392.4257051967895,
    394.2978805629408, 396.0140413152461, 397.8862166813974, 399.9144066613946,
    401.318538186008, 402.87868432446743, 404.5948450767728, 406.15499121523214,
    408.3391958090753, 410.3673857890725, 413.33166345214534, 416.45195572906414,
    420.6643503029045, 425.5008033321286, 430.6492855890446, 436.2658116874984,
    441.4142939444144, 446.5627762013304, 451.5552438444005, 456.07966764593266,
    459.51198915054334, 462.63228142746215, 465.2845298628431, 467.93677829822406,
    471.05707057514286, 474.80142130744537, 479.79388895051545, 484.0062835243558,
    486.97056118742864, 489.15476578127175, 491.80701421665276, 494.08482757880347,
    496.0194087904931, 498.67165722587407, 502.10397873048476, 506.00434407663323,
    510.99681171970326, 516.3013085904652, 522.073849302765, 527.6903754012187,
    532.214799202751, 535.1790768658238, 537.0512522319751, 539.2354568258182,
    541.4196614196615, 546.2561144488855, 550.31249440888, 555.4609766657959,
    561.0775027642497, 565.9139557934739, 570.4383795950062, 574.1827303273086,
    576.8349787626896, 578.5511395149949, 579.9552710396083, 581.2033879503758,
    581.9834610196056, 583.0755633165272, 584.0116509996028, 584.7917240688325,
    585.5717971380622, 586.8199140488297, 588.0680309595972, 589.4721624842107,
    591.344337850362, 595.2447031965104, 598.5210100872752, 600.2371708395805,
    601.1732585226562, 602.1093462057318, 603.2014485026533, 604.137536185729,
    604.4495654134208, 605.2296384826504, 606.1657261657261, 606.477755393418,
    607.2578284626477, 607.7258723041855, 608.0379015318774, 609.130003828799,
    610.2221061257205, 611.1581938087962, 611.9382668780258, 613.0303691749475,
    614.1224714718691, 614.9025445410987, 617.2427637487879, 620.2070414118607,
    623.9513921441633, 628.6318305595414, 633.6242982026115, 638.6167658456815,
    642.9851750333678, 648.4456865179757, 653.7501833887376, 659.0546802594995,
    664.9832355856452, 670.1317178425612, 675.7482439410151, 680.5846969702391,
    685.8891938410011, 691.5057199394548, 696.8102168102168
])

values = np.array([
    0.0196400097922671, 0.04783516511041985, 0.0784920367311468, 0.10914730820198826,
    0.1341872536867903, 0.14325570312595337, 0.1686425611059531, 0.20197640347779366,
    0.2322008345919374, 0.26026626347293624, 0.2894835335799191, 0.32080225386249483,
    0.35065019637772166, 0.38094496741392003, 0.41216137144589393, 0.44169357771952766,
    0.4738681572178195, 0.5000441900719935, 0.5308209662574814, 0.5556747914513039,
    0.5792670441882674, 0.5972408944605421, 0.6090366927030503, 0.6172666636021592,
    0.61967188445409, 0.6114491586780743, 0.5988708604723432, 0.5811598071971433,
    0.5570055446740206, 0.5296925405581568, 0.5021962583223007, 0.47291586230233107,
    0.44668812555744886, 0.4220558627046114, 0.41325681627813216, 0.43261830764147713,
    0.46092753935942055, 0.48821841568829505, 0.5191662478965536, 0.5507452325580221,
    0.5773442040763633, 0.6062489382415238, 0.6335318271555527, 0.6547261902034839,
    0.6689915668414277, 0.6734834138541272, 0.6691582442722386, 0.6682374380205703,
    0.6491328027803598, 0.624565053907037, 0.5968057870374829, 0.5643000755697696,
    0.5379812102889024, 0.5251527197588365, 0.5448371413709688, 0.5558943548748922,
    0.5460817557314313, 0.5323516162997162, 0.5126992087974477, 0.49196369984136423,
    0.4631149442530349, 0.43277942273308345, 0.4009171715381167, 0.3692699404840274,
    0.33977851136331194, 0.3067204548242546, 0.2746112071598632, 0.2448320177513874,
    0.21361402690411013, 0.18196679585002096, 0.15118284565921258, 0.11730627244992409,
    0.08933118536454954, 0.07237333026026249, 0.097856917300847, 0.13110203136153098,
    0.15873181943617531, 0.19027514742435958, 0.22118620285441404, 0.25485895697202454,
    0.28378046604181906, 0.3155819782140449, 0.3500029090798321, 0.3747710779962863,
    0.3865252901212122, 0.4137210375393877, 0.44105986502650063, 0.4750211795068142,
    0.5076587966634305, 0.5386266040760942, 0.5680540938742976, 0.596763783102986,
    0.6058338326920346, 0.6314772747061902, 0.6602365285775849, 0.6875684030324569,
    0.7092381407673343, 0.7235542373077126, 0.7297187258447684, 0.7325534167663121,
    0.7375362581108641, 0.7387824459520473, 0.7454569111473651, 0.7501458303571369,
    0.7571688081975383, 0.7612718147472548, 0.7657129418575199, 0.7704570129000154,
    0.7672037481676034, 0.7703403619733553, 0.7723531105069565
])

glasses_transmittance = SpectralDistribution(values, wavelengths)

In [6]:
# Define Hunt-Pointer-Estevez transformation matrices manually
M_XYZ_TO_LMS = np.array([
    [0.4002, 0.7076, -0.0808],    # XYZ to LMS matrix
    [-0.2263, 1.1653, 0.0457],
    [0.0, 0.0, 0.9182]
])
M_LMS_TO_XYZ = np.linalg.inv(M_XYZ_TO_LMS)  # LMS to XYZ matrix


def simulate_glasses_spectral(image_path, output_path, glasses_transmittance):
    """
    Simulate color-correcting glasses using spectral transmittance data.
    """
    # Load image and convert to float32
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_float = img.astype(np.float32) / 255.0

    # 1. Define CIE Standard Observer and Illuminant (D65)
    cmfs = MSDS_CMFS["CIE 1931 2 Degree Standard Observer"]
    illuminant = SDS_ILLUMINANTS["D65"]

    # 2. Compute Glasses' Transmittance Matrix
    XYZ_glasses = sd_to_XYZ(
        glasses_transmittance,
        cmfs,
        illuminant,
        method="Integration"  # Add this parameter
    )
    XYZ_glasses /= XYZ_glasses[1]  # Normalize to luminance (Y=1)

    # 3. Convert Image to LMS (Human Cone Response)
    # RGB -> XYZ
    XYZ = np.tensordot(
        img_float, RGB_COLOURSPACE_sRGB.matrix_RGB_to_XYZ, axes=(-1, -1))

    # XYZ -> LMS using manual matrix
    LMS = np.dot(XYZ, M_XYZ_TO_LMS.T)

    # 4. Apply Glasses' Transmittance in LMS Space
    L_ratio, M_ratio, S_ratio = XYZ_glasses[0], XYZ_glasses[1], XYZ_glasses[2]
    LMS_filtered = LMS * np.array([L_ratio, M_ratio, S_ratio])

    # 5. Convert Back to RGB
    # LMS -> XYZ
    XYZ_filtered = np.dot(LMS_filtered, M_LMS_TO_XYZ.T)

    # XYZ -> RGB
    RGB_filtered = np.tensordot(
        XYZ_filtered, RGB_COLOURSPACE_sRGB.matrix_XYZ_to_RGB, axes=(-1, -1))

    # Manual normalization to [0, 1] (replaces normalise_maximum)
    RGB_filtered = RGB_filtered / \
        np.max(RGB_filtered, axis=(0, 1), keepdims=True)
    # Ensure values stay within [0, 1]
    RGB_filtered = np.clip(RGB_filtered, 0.0, 1.0)

    # Save result
    RGB_filtered = (RGB_filtered * 255).astype(np.uint8)
    cv2.imwrite(output_path, cv2.cvtColor(RGB_filtered, cv2.COLOR_RGB2BGR))

In [7]:
# ====== ENHANCED ANALYSIS FUNCTIONS ======
def compute_color_gradient(img):
    """Compute average gradient magnitude in LAB color channels"""
    lab = rgb2lab(img)
    a, b = lab[:, :, 1], lab[:, :, 2]

    # Compute gradients
    dx_a, dy_a = np.gradient(a)
    dx_b, dy_b = np.gradient(b)

    # Calculate gradient magnitudes
    mag_a = np.sqrt(dx_a**2 + dy_a**2)
    mag_b = np.sqrt(dx_b**2 + dy_b**2)

    return np.mean(mag_a), np.mean(mag_b)


def generate_comparison_plots(original, daltonized, glass_effect, deficiency, img_num):
    """Generate and save side-by-side comparison images"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))

    # Original image
    axes[0].imshow(original)
    axes[0].set_title("Original")
    axes[0].axis('off')

    # Daltonized image
    axes[1].imshow(daltonized)
    axes[1].set_title(f"Daltonized ({deficiency})")
    axes[1].axis('off')

    # Glass effect image
    axes[2].imshow(glass_effect)
    axes[2].set_title(f"Glass Effect ({deficiency})")
    axes[2].axis('off')

    plt.tight_layout()
    os.makedirs("comparisons", exist_ok=True)
    plt.savefig(
        f"comparisons/{deficiency}_{img_num}_comparison.png", bbox_inches='tight')
    plt.close()

In [8]:
# ====== METRIC COMPUTATION FUNCTIONS ======
def compute_gradients(image, method='scharr'):
    """Compute gradients using different operators with grayscale conversion"""
    if len(image.shape) == 3:  # Convert color to grayscale
        image = cv2.cvtColor(
            (image * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    if method.lower() == 'scharr':
        dx = cv2.Scharr(image, cv2.CV_64F, 1, 0)
        dy = cv2.Scharr(image, cv2.CV_64F, 0, 1)
    else:  # Default to Sobel
        dx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
        dy = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    return dx, dy


def gradient_magnitude(grad_x, grad_y):
    """Compute gradient magnitude"""
    return np.sqrt(grad_x**2 + grad_y**2)


def gradient_orientation(grad_x, grad_y):
    """Compute orientation in degrees (0-360) with epsilon smoothing"""
    eps = 1e-6  # Avoid division by zero
    return np.degrees(np.arctan2(grad_y + eps, grad_x + eps)) % 360


def compute_additional_metrics(img1, img2):
    """Compute metrics including new color gradient metric"""
    # Convert to grayscale for SSIM
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_RGB2GRAY)
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_RGB2GRAY)

    # PSNR
    psnr_val = peak_signal_noise_ratio(img1, img2, data_range=255)

    # SSIM
    ssim_val = structural_similarity(img1_gray, img2_gray, data_range=255,
                                     win_size=11, gaussian_weights=True)

    # Color difference in LAB space
    lab1 = rgb2lab(img1)
    lab2 = rgb2lab(img2)
    delta_e = deltaE_ciede2000(lab1, lab2)

    # NEW: Color gradient metrics
    color_grad1 = compute_color_gradient(img1)
    color_grad2 = compute_color_gradient(img2)

    return {
        'PSNR': psnr_val,
        'SSIM': ssim_val,
        'DeltaE_mean': np.mean(delta_e),
        'DeltaE_std': np.std(delta_e),
        'ColorGradient_mean': (np.mean(color_grad1) + np.mean(color_grad2)) / 2,
        'ColorGradient_diff': np.mean(color_grad1) - np.mean(color_grad2)
    }


def quantify_differences(mag_diff, orient_diff):
    """Quantify gradient differences with statistical analysis"""
    return {
        'Magnitude_Mean': np.nanmean(mag_diff),
        'Magnitude_Std': np.nanstd(mag_diff),
        'Orientation_Mean': np.nanmean(orient_diff),
        'Orientation_Std': np.nanstd(orient_diff),
        'Magnitude_Max': np.nanmax(np.abs(mag_diff)),
        'Orientation_Max': np.nanmax(orient_diff)
    }


def structural_comparison(original, processed):
    """Compare structural changes using histogram analysis"""

    # Convert to grayscale
    gray_orig = cv2.cvtColor(
        (original * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    gray_proc = cv2.cvtColor(
        (processed * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)

    # Histogram comparison
    hist_orig = cv2.calcHist([gray_orig], [0], None, [256], [0, 256])
    hist_proc = cv2.calcHist([gray_proc], [0], None, [256], [0, 256])
    hist_corr = cv2.compareHist(hist_orig, hist_proc, cv2.HISTCMP_CORREL)

    # Edge preservation
    edges_orig = cv2.Canny(gray_orig, 100, 200)
    edges_proc = cv2.Canny(gray_proc, 100, 200)
    edge_similarity = np.sum(edges_orig == edges_proc) / edges_orig.size

    return {
        'Histogram_Correlation': hist_corr,
        'Edge_Preservation': edge_similarity
    }

In [9]:
def plot_comparison(original, processed, mag_diff, orient_diff, metrics, label, output_path):
    """Generate and save comparison plot with metrics"""
    # Convert images to proper format
    if original.dtype != np.uint8:
        original = (np.clip(original, 0, 1) * 255).astype(np.uint8)
    if processed.dtype != np.uint8:
        processed = (np.clip(processed, 0, 1) * 255).astype(np.uint8)

    fig = plt.figure(figsize=(20, 15))

    # Original image
    ax1 = fig.add_subplot(2, 3, 1)
    ax1.imshow(original)
    ax1.set_title('Original Image')
    ax1.axis('off')

    # Processed image
    ax2 = fig.add_subplot(2, 3, 2)
    ax2.imshow(processed)
    ax2.set_title(label)
    ax2.axis('off')

    # Magnitude difference
    ax3 = fig.add_subplot(2, 3, 3)
    mag_plot = ax3.imshow(mag_diff, cmap='coolwarm', vmin=-1, vmax=1)
    plt.colorbar(mag_plot, ax=ax3)
    ax3.set_title('Gradient Magnitude Difference')
    ax3.axis('off')

    # Orientation difference
    ax4 = fig.add_subplot(2, 3, 4)
    orient_plot = ax4.imshow(orient_diff, cmap='viridis', vmin=0, vmax=180)
    plt.colorbar(orient_plot, ax=ax4)
    ax4.set_title('Gradient Orientation Difference')
    ax4.axis('off')

    # Metric visualization
    ax5 = fig.add_subplot(2, 3, 5)
    metrics_display = metrics.copy()
    metrics_display.pop('DeltaE_std', None)  # Simplify for display
    ax5.barh(range(len(metrics_display)), list(metrics_display.values()),
             color=['#1f77b4', '#ff7f0e', '#2ca02c'])
    ax5.set_yticks(range(len(metrics_display)))
    ax5.set_yticklabels(list(metrics_display.keys()))
    ax5.set_title('Quality Metrics Comparison')

    # Vector field for orientation
    ax6 = fig.add_subplot(2, 3, 6)
    y, x = np.mgrid[0:orient_diff.shape[0]:10, 0:orient_diff.shape[1]:10]
    angles = np.deg2rad(orient_diff[::10, ::10])
    ax6.quiver(x, y, np.cos(angles), np.sin(angles), scale=50, width=0.002)
    ax6.set_title('Orientation Vector Field')
    ax6.axis('off')

    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

In [10]:
def compute_differences(img1, img2, gradient_method='scharr'):
    """Compute gradient differences between two images"""
    # Convert to grayscale
    gray1 = cv2.cvtColor((img1 * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
    gray2 = cv2.cvtColor((img2 * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)

    grad_x1, grad_y1 = compute_gradients(gray1, gradient_method)
    grad_x2, grad_y2 = compute_gradients(gray2, gradient_method)

    # Magnitude difference
    mag1 = gradient_magnitude(grad_x1, grad_y1)
    mag2 = gradient_magnitude(grad_x2, grad_y2)
    mag_diff = mag1 - mag2

    # Orientation difference (handling 180-degree periodicity)
    orient1 = gradient_orientation(grad_x1, grad_y1)
    orient2 = gradient_orientation(grad_x2, grad_y2)
    orient_diff = np.abs(orient1 - orient2)
    orient_diff = np.minimum(orient_diff, 180 - orient_diff)

    return mag_diff, orient_diff


def compute_metrics(original, processed):
    """Compute all metrics without plotting"""

    # This is the correct function that uses compute_differences

    mag_diff, orient_diff = compute_differences(original, processed)

    metrics = {





        **compute_additional_metrics(original, processed),





        **quantify_differences(mag_diff, orient_diff),





        **structural_comparison(original, processed)





    }

    return metrics

In [11]:
def process_test_set():
    """Process all images with enhanced metric tracking and comparison plots"""
    os.makedirs(GLASS_OUTPUT_DIR, exist_ok=True)

    # Create dedicated directory for comparison plots
    COMPARISON_PLOT_DIR = "comparison_plots"
    os.makedirs(COMPARISON_PLOT_DIR, exist_ok=True)

    # Initialize metrics storage
    all_metrics = {method: {deftype: [] for deftype in DEFICIENCY_TYPES}
                   for method in METHODS}
    total_images = 0

    for deficiency in DEFICIENCY_TYPES:
        type_dir = os.path.join(TEST_SET_DIR, deficiency)
        glass_type_dir = os.path.join(GLASS_OUTPUT_DIR, deficiency)
        comp_type_dir = os.path.join(
            COMPARISON_PLOT_DIR, deficiency)  # New directory for plots

        # Create all necessary directories
        os.makedirs(glass_type_dir, exist_ok=True)
        # Create directory for comparison plots
        os.makedirs(comp_type_dir, exist_ok=True)

        # Get all original images for this deficiency type
        orig_images = glob.glob(os.path.join(type_dir, "original_*.png"))

        for orig_path in tqdm(orig_images, desc=f"Processing {deficiency} images"):
            try:
                # Extract image number from filename
                img_num = os.path.basename(orig_path).split("_")[
                    1].split(".")[0]

                # Construct paths
                daltonized_path = os.path.join(
                    type_dir, f"generated_{img_num}.png")
                glass_path = os.path.join(
                    glass_type_dir, f"glass_{img_num}.png")

                # Generate glass effect if missing
                if not os.path.exists(glass_path):
                    simulate_glasses_spectral(
                        orig_path, glass_path, glasses_transmittance)

                # Load images
                original = load_image(orig_path)
                daltonized = load_image(daltonized_path)
                glass = load_image(glass_path)

                # Compute metrics
                metrics_daltonized = compute_metrics(original, daltonized)
                metrics_glass = compute_metrics(original, glass)

                # Compute gradient differences with Scharr operator
                mag_diff_dalton, orient_diff_dalton = compute_differences(
                    original, daltonized)
                mag_diff_glass, orient_diff_glass = compute_differences(
                    original, glass)

                # Generate and save comparison plots in the new directory
                plot_comparison(
                    original,
                    daltonized,
                    mag_diff_dalton,
                    orient_diff_dalton,
                    metrics_daltonized,
                    'Daltonized',
                    os.path.join(comp_type_dir, f"daltonized_comparison_{img_num}.png"))

                plot_comparison(
                    original,
                    glass,
                    mag_diff_glass,
                    orient_diff_glass,
                    metrics_glass,
                    'Glass Effect',
                    os.path.join(comp_type_dir, f"glass_comparison_{img_num}.png"))

                # Store metrics
                all_metrics['daltonized'][deficiency].append(
                    metrics_daltonized)
                all_metrics['glass_effect'][deficiency].append(metrics_glass)

                total_images += 1
            except Exception as e:
                print(f"Error processing {orig_path}: {str(e)}")

    print(
        f"\nProcessed {total_images} images across {len(DEFICIENCY_TYPES)} deficiency types")
    print(f"Glass effect images saved to: {GLASS_OUTPUT_DIR}")
    print(f"Comparison plots saved to: {COMPARISON_PLOT_DIR}")
    return all_metrics

In [12]:
def aggregate_metrics(all_metrics):
    """Average metrics with deficiency-specific breakdowns"""
    # Overall aggregation
    overall = {method: [] for method in METHODS}
    for method in METHODS:
        for deftype in DEFICIENCY_TYPES:
            overall[method].extend(all_metrics[method][deftype])

    overall_agg = {}
    for method in METHODS:
        agg = {}
        for key in overall[method][0].keys():
            values = [m[key] for m in overall[method]]
            agg[key] = {
                'mean': np.mean(values),
                'std': np.std(values),
                'min': np.min(values),
                'max': np.max(values)
            }
        overall_agg[method] = agg

    # Deficiency-specific aggregation
    type_agg = {}
    for deftype in DEFICIENCY_TYPES:
        type_agg[deftype] = {}
        for method in METHODS:
            agg = {}
            for key in all_metrics[method][deftype][0].keys():
                values = [m[key] for m in all_metrics[method][deftype]]
                agg[key] = {
                    'mean': np.mean(values),
                    'std': np.std(values),
                    'min': np.min(values),
                    'max': np.max(values)
                }
            type_agg[deftype][method] = agg

    return {
        'overall': overall_agg,
        'by_deficiency': type_agg
    }

In [13]:
def enhancement_comparison(aggregated_metrics, deficiency_type="overall"):
    key_metrics = ['PSNR', 'SSIM', 'DeltaE_mean',
                   'ColorGradient_mean', 'Edge_Preservation']
    metric_names = {
        'PSNR': 'PSNR (dB)',
        'SSIM': 'Structural Similarity',
        'DeltaE_mean': 'Color Shift (ΔE)',
        'ColorGradient_mean': 'Color Gradient Strength',
        'Edge_Preservation': 'Edge Preservation'
    }

    plt.figure(figsize=(10, 6))

    # Access the correct level of the metrics dictionary
    if deficiency_type == 'overall':
        metrics_data = aggregated_metrics['overall']
    else:
        metrics_data = aggregated_metrics['by_deficiency'][deficiency_type]

    # Calculate enhancement percentages
    enhancements = {}
    for metric in key_metrics:
        dal_val = metrics_data['daltonized'][metric]['mean']
        glass_val = metrics_data['glass_effect'][metric]['mean']

        enhancement = (dal_val - glass_val) / glass_val * 100
        enhancements[metric] = enhancement

    # Sort metrics by enhancement value
    sorted_metrics = sorted(enhancements.items(),
                            key=lambda x: x[1], reverse=True)
    metrics, values = zip(*sorted_metrics)

    # Create bar plot
    bars = plt.barh([metric_names[m] for m in metrics], values,
                    color=["#3bad3b" if v > 0 else "#ca3e25" for v in values])

    # Add data labels with improved positioning
    # Calculate max_abs earlier for padding
    max_abs = max(abs(v) for v in values)

    for bar in bars:
        width = bar.get_width()
        # Position labels inside bars for negative values
        if width < 0:
            # Place label at the end of the bar with padding
            label_x = width - 0.01 * max_abs
            ha = 'right'
            color = 'white'  # White text for better contrast
        else:
            label_x = width - 0.01 * max_abs
            ha = 'right'
            color = 'black'

        plt.text(label_x, bar.get_y() + bar.get_height()/2,
                 f'{width:.1f}%', ha=ha, va='center',
                 fontsize=10, color=color, fontweight='bold')

    # Set symmetric x-limits with proper padding for negative values
    padding = max_abs * 0.2
    plt.xlim(-max_abs - padding, max_abs + padding)

    # Add reference line and annotations
    plt.axvline(0, color='black', linewidth=0.8)
    plt.title(f"Daltonized vs Glass Effect - {deficiency_type.capitalize()}",
              fontsize=16, pad=15)
    plt.xlabel("Performance Improvement (%)", fontsize=12)
    plt.grid(axis='x', linestyle='--', alpha=0.7)

    # Create output directory if it doesn't exist
    output_dir = "enhancement_comparisons"
    os.makedirs(output_dir, exist_ok=True)

    # Save with deficiency-specific filename
    output_path = os.path.join(
        output_dir, f"enhancement_{deficiency_type}.png")
    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

In [14]:
def generate_comparative_visualizations(aggregated_metrics):
    """Create attractive visualizations for each deficiency type"""
    # Define metrics including our new color gradient metric
    key_metrics = ['PSNR', 'SSIM', 'DeltaE_mean',
                   'ColorGradient_mean', 'Edge_Preservation']
    metric_names = {
        'PSNR': 'PSNR (dB)',
        'SSIM': 'Structural Similarity',
        'DeltaE_mean': 'Color Shift (ΔE)',
        'ColorGradient_mean': 'Color Gradient Strength',
        'Edge_Preservation': 'Edge Preservation'
    }

    # Create professional color scheme
    method_colors = {'daltonized': '#4c72b0', 'glass_effect': '#55a868'}

    # ==================================================================
    # Enhanced Bar Chart Comparison
    # ==================================================================
    plt.figure(figsize=(16, 12))
    plt.suptitle("Performance Comparison: Daltonized vs Glass Effect",
                 fontsize=18, fontweight='bold', y=0.98)

    # Overall comparison
    plt.subplot(3, 2, 1)
    for i, method in enumerate(['daltonized', 'glass_effect']):
        values = [aggregated_metrics['overall'][method][metric]['mean']
                  for metric in key_metrics]
        errors = [aggregated_metrics['overall'][method][metric]['std']
                  for metric in key_metrics]

        # Position bars with slight offset for dual display
        positions = np.arange(len(key_metrics)) + i * 0.35
        plt.bar(positions, values, yerr=errors, width=0.35,
                color=method_colors[method], label=method.capitalize(),
                edgecolor='white', linewidth=1.5, alpha=0.9,
                error_kw=dict(elinewidth=1, ecolor='black', capsize=4))

    plt.title("Overall Performance", fontsize=14, pad=15)
    plt.xticks(np.arange(len(key_metrics)) + 0.17,
               [metric_names[m] for m in key_metrics], rotation=15, ha='right')
    plt.ylabel("Metric Value", fontsize=12)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.legend(frameon=True, facecolor='white')

    # Deficiency-specific comparisons
    for idx, deftype in enumerate(DEFICIENCY_TYPES, 2):
        plt.subplot(3, 2, idx)
        for i, method in enumerate(['daltonized', 'glass_effect']):
            values = [aggregated_metrics['by_deficiency'][deftype][method][metric]['mean']
                      for metric in key_metrics]
            errors = [aggregated_metrics['by_deficiency'][deftype][method][metric]['std']
                      for metric in key_metrics]

            positions = np.arange(len(key_metrics)) + i * 0.35
            plt.bar(positions, values, yerr=errors, width=0.35,
                    color=method_colors[method], label=method.capitalize(),
                    edgecolor='white', linewidth=1.5, alpha=0.9,
                    error_kw=dict(elinewidth=1, ecolor='black', capsize=4))

        plt.title(f"{deftype.capitalize()} Performance", fontsize=14, pad=15)
        plt.xticks(np.arange(len(key_metrics)) + 0.17,
                   [metric_names[m] for m in key_metrics], rotation=15, ha='right')
        plt.grid(axis='y', linestyle='--', alpha=0.7)
        plt.ylim(0, 12)  # Consistent scale for comparison

    plt.tight_layout(pad=3.0)
    plt.savefig("deficiency_comparison.png", dpi=300, bbox_inches='tight')
    plt.close()

    enhancement_comparison(aggregated_metrics, deficiency_type='overall')

In [15]:
# ====== Enhanced Reporting ======
def generate_final_report(aggregated_metrics):
    """Generate comprehensive reports with deficiency breakdowns"""
    # Overall report
    print("\n" + "="*70)
    print("OVERALL REPORT (All Deficiency Types)")
    print("="*70)
    print_report_table(aggregated_metrics['overall'])

    # Deficiency-specific reports
    for deftype in DEFICIENCY_TYPES:
        print("\n" + "="*70)
        print(f"{deftype.upper()} REPORT")
        print("="*70)
        print_report_table(aggregated_metrics['by_deficiency'][deftype])
        enhancement_comparison(aggregated_metrics, deficiency_type=deftype)

    # Visual comparisons
    generate_comparative_visualizations(aggregated_metrics)


def print_report_table(metrics_dict):
    """Print formatted metrics table with extended metrics"""
    print(f"{'METRIC':<25} | {'DALTONIZED':^15} | {'GLASS EFFECT':^15} | {'DIFFERENCE':^10}")
    print("-"*70)

    # Updated key metrics to match radar plot categories
    key_metrics = [
        'PSNR', 'SSIM', 'DeltaE_mean', 'ColorGradient_mean', 'Edge_Preservation',
        'Magnitude_Mean', 'Magnitude_Std', 'Orientation_Mean', 'Orientation_Std',
        'Magnitude_Max', 'Orientation_Max', 'Histogram_Correlation'
    ]

    for metric in key_metrics:
        dal = metrics_dict['daltonized'][metric]
        glass = metrics_dict['glass_effect'][metric]

        # Format values based on metric type
        if metric in ['Magnitude_Mean', 'Magnitude_Std', 'Orientation_Mean',
                      'Orientation_Std', 'Magnitude_Max', 'Orientation_Max']:
            # Format gradient metrics with 2 decimal places
            dal_val = f"{dal['mean']:.2f} ± {dal['std']:.2f}"
            glass_val = f"{glass['mean']:.2f} ± {glass['std']:.2f}"
            diff = dal['mean'] - glass['mean']
            diff_sign = "+" if diff > 0 else ""
            diff_str = f"{diff_sign}{diff:.2f}"
        else:
            # Format other metrics with 4 decimal places
            dal_val = f"{dal['mean']:.4f} ± {dal['std']:.2f}"
            glass_val = f"{glass['mean']:.4f} ± {glass['std']:.2f}"
            diff = dal['mean'] - glass['mean']
            diff_sign = "+" if diff > 0 else ""
            diff_str = f"{diff_sign}{diff:.4f}"

        print(f"{metric:<25} | {dal_val} | {glass_val} | {diff_str}")


def generate_recommendations(aggregated_metrics):
    """Generate enhanced recommendations based on quantitative results"""
    weights = {
        # Structural similarity (less critical for accessibility)
        'SSIM': 0.1,
        'Edge_Preservation': 0.1,     # Edge retention
        'DeltaE_mean': 0.4,           # Higher = better correction (CRUCIAL)
        'ColorGradient_mean': 0.4,    # Higher = better separation (CRUCIAL)
        'PSNR': 0.0                   # Not relevant for accessibility
    }

    recommendations = {}
    detailed_reasons = {}

    # Overall recommendation
    overall_scores = {}
    for method in METHODS:
        score = 0
        for metric, weight in weights.items():
            value = aggregated_metrics['overall'][method][metric]['mean']
            score += weight * value
        overall_scores[method] = score

    best_overall = max(overall_scores, key=overall_scores.get)
    recommendations['overall'] = best_overall

    # Deficiency-specific recommendations
    for deftype in DEFICIENCY_TYPES:
        type_scores = {}
        reasons = []

        for method in METHODS:
            score = 0
            for metric, weight in weights.items():
                value = aggregated_metrics['by_deficiency'][deftype][method][metric]['mean']
                score += weight * value
            type_scores[method] = score

        best_method = max(type_scores, key=type_scores.get)
        recommendations[deftype] = best_method

        # Build detailed reasons
        daltonized = aggregated_metrics['by_deficiency'][deftype]['daltonized']
        glass = aggregated_metrics['by_deficiency'][deftype]['glass_effect']

        deltaE_d = daltonized['DeltaE_mean']['mean']
        deltaE_g = glass['DeltaE_mean']['mean']
        cg_d = daltonized['ColorGradient_mean']['mean']
        cg_g = glass['ColorGradient_mean']['mean']

        deltaE_increase = (deltaE_d - deltaE_g) / deltaE_g * 100
        cg_increase = (cg_d - cg_g) / cg_g * 100

        reasons.append(f"- ΔE (color shift): {deltaE_d:.2f} vs {deltaE_g:.2f} "
                       f"({deltaE_increase:.1f}% higher)")
        reasons.append(f"- Color gradient: {cg_d:.2f} vs {cg_g:.2f} "
                       f"({cg_increase:.1f}% higher)")
        reasons.append(
            f"- Visual evidence: See comparisons/{deftype}_*_comparison.png in the comparisons folder!!!")

        detailed_reasons[deftype] = reasons

    # Print enhanced recommendations
    print("\n" + "="*70)
    print("ACCESSIBILITY RECOMMENDATIONS")
    print("="*70)
    print(f"Overall best method: {recommendations['overall'].upper()}")

    for deftype in DEFICIENCY_TYPES:
        print(
            f"\n{deftype.capitalize()} recommendation: {recommendations[deftype].upper()}")
        for reason in detailed_reasons[deftype]:
            print(reason)

    print("\n" + "="*70)
    print("KEY OBSERVATIONS")
    print("="*70)
    print("1. Daltonization achieved higher ΔE on average.")
    print("   - Signifies more effective color transformations for accessibility")
    print("2. Color gradient strength was higher in daltonized images.")
    print("   - Indicates superior color separation in LAB space")
    print("3. Glass effect showed minimal ΔE improvement over originals.")
    print("   - Merely filters light without addressing color confusion points")
    print("4. Visual comparisons demonstrate clearer color boundaries.")
    print("   - Critical for interpreting medical/images/data visualizations")

    return recommendations

In [16]:
if __name__ == "__main__":
    # Process all images
    print(
        f"Starting batch processing for {len(DEFICIENCY_TYPES)} deficiency types...")
    all_metrics = process_test_set()

    # Aggregate results
    aggregated = aggregate_metrics(all_metrics)

    # Generate final report
    generate_final_report(aggregated)

    # Generate recommendations
    recommendations = generate_recommendations(aggregated)

    # Print recommendations
    print("\n" + "="*70)
    print("METHOD RECOMMENDATIONS")
    print("="*70)
    print(f"Overall best method: {recommendations['overall'].upper()}")
    for deftype in DEFICIENCY_TYPES:
        print(f"{deftype.capitalize()}: {recommendations[deftype].upper()}")

    # Save detailed results
    results_data = []
    for deftype in DEFICIENCY_TYPES:
        for method in METHODS:
            for metric, stats in aggregated['by_deficiency'][deftype][method].items():
                results_data.append({
                    'Deficiency': deftype,
                    'Method': method,
                    'Metric': metric,
                    'Mean': stats['mean'],
                    'Std': stats['std'],
                    'Min': stats['min'],
                    'Max': stats['max']
                })

    results_df = pd.DataFrame(results_data)
    results_df.to_csv("detailed_results.csv", index=False)
    print("\nDetailed results saved to detailed_results.csv")

Starting batch processing for 3 deficiency types...


Processing tritanopia images: 100%|██████████| 112/112 [09:57<00:00,  5.33s/it]



Processed 336 images across 3 deficiency types
Glass effect images saved to: test_results_glass
Comparison plots saved to: comparison_plots

OVERALL REPORT (All Deficiency Types)
METRIC                    |   DALTONIZED    |  GLASS EFFECT   | DIFFERENCE
----------------------------------------------------------------------
PSNR                      | 73.7342 ± 3.83 | 75.7319 ± 5.04 | -1.9977
SSIM                      | 0.9998 ± 0.00 | 0.9999 ± 0.00 | -0.0000
DeltaE_mean               | 5.8930 ± 2.39 | 5.6061 ± 2.98 | +0.2870
ColorGradient_mean        | 1.9211 ± 0.82 | 1.6643 ± 0.81 | +0.2568
Edge_Preservation         | 0.9524 ± 0.03 | 0.9876 ± 0.01 | -0.0352
Magnitude_Mean            | 16.77 ± 14.84 | 10.43 ± 19.03 | +6.34
Magnitude_Std             | 95.75 ± 28.99 | 31.77 ± 16.44 | +63.98
Orientation_Mean          | 7.96 ± 2.22 | 2.55 ± 1.27 | +5.42
Orientation_Std           | 38.83 ± 4.68 | 20.62 ± 4.54 | +18.21
Magnitude_Max             | 1021.82 ± 247.68 | 268.93 ± 116.56 | +752.89